<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Nuclear Incidents
  </div> 

  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Data Preparation
  </div> 


  <div style=" float:left; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 
  
  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jan 2023
  </div> 

<a id="TOC"></a>

***
<div style="font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Table of Content
  </div> 

1. [Corpus Import](#corpus)
2. [Metadata Extraction](#metadata)
3. [Text Segmentation](#segmentation)



[Bottom](#bottom)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import warnings
import unicodedata
import re
import copy

# data 
import numpy as np
import pandas as pd

# nlp
from nltk.tokenize.punkt import PunktSentenceTokenizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as en_stop
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

# visualization
from IPython.core.display import display, HTML
from tqdm.notebook import trange


warnings.filterwarnings("ignore")
print('python version :', sys.version)

In [ ]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data_raw = os.path.join(path_to_repo, 'data', 'raw')
path_to_data_prc = os.path.join(path_to_repo, 'data', 'processed')

In [ ]:
sys.path.insert(0, os.path.join(path_to_repo, 'src'))

In [ ]:
from tmtools.tfidf import compute_gensim_tfidf_similarity_matrix
from tmtools.span import get_maximal_spans

In [ ]:
# load relevant nlp models:

# 450 MB transformer-based model
nlp_en = spacy.load('en_core_web_trf')

# Merge multi-word entities into single tokens
nlp_en.add_pipe("merge_entities")

In [ ]:
# 400 MB transformer-based model
nlp = spacy.load('fr_dep_news_trf', disable = ['ner'])

<a id="corpus"></a>

# 1. Corpus Import

In [ ]:
df_incidents = pd.read_excel(os.path.join(path_to_data_raw, 'data_nuclear_incidents.xlsx'))
df_incidents = df_incidents[['text']]
df_incidents.shape

In [ ]:
df_incidents.head(2)

<a id="metadata"></a>

# 2. Metadata Extraction

[Table of content](#TOC)



### Find titles

In [ ]:
df_incidents['Doc_id'] = df_incidents.index.tolist()
df_incidents['title']  = df_incidents['text'].apply(lambda t: t.split('\n')[0])
df_incidents['text']   = df_incidents['text'].apply(lambda t: '\n'.join(t.split('\n')[1:]))

df_incidents = df_incidents[['Doc_id', 'text', 'title']]

In [ ]:
df_incidents.head(2)

### Find dates and locations

In [ ]:
# an example
text = df_incidents.text[1]
doc = nlp_en(text)

spacy.displacy.render(doc, style = "ent", jupyter = True)

In [ ]:
titles = df_incidents['title'].tolist()
texts  = df_incidents.text.tolist()

common_entities = []
for i in trange(len(texts)):
    title = titles[i]
    text = texts[i]
    ents = nlp_en(title + '\n' + text).ents
    common_entities.append(ents)

In [ ]:
dates = [[ent.text for ent in ents if ent.label_ == 'DATE'] for ents in common_entities]
dates = [ds[0].replace('2021à', '2021') if len(ds)>0 else None for ds in dates]

In [ ]:
df_incidents['date'] = dates

In [ ]:
locations = [[ent for ent in ents if ent.label_ == 'GPE'] for ents in common_entities]

In [ ]:
unique_locations = [loc.text for locs in locations for loc in locs]
unique_locations += ['Saint-Laurent-des-Eaux']
unique_locations = set(unique_locations)
unique_locations - set(['Areva NC', 'Aucun', 'Saint'])
unique_locations

In [ ]:
infered_locations = [[(loc, text.count(loc)) for loc in unique_locations] for text in df_incidents.text]
infered_locations = [sorted(locs, key = lambda lc: lc[1], reverse = True)[0] for locs in infered_locations]
infered_locations = [lc[0] if lc[1]>0 else 'Inconnu' for lc in infered_locations]

In [ ]:
len([loc for loc in infered_locations if loc == 'Inconnu'])

In [ ]:
infered_locations

In [ ]:
df_incidents['location'] = infered_locations

In [ ]:
df_incidents = df_incidents[['Doc_id', 'title', 'date', 'location', 'text']]

### Export result

In [ ]:
df_incidents.to_excel(os.path.join(path_to_data_prc, 'source_texts.xlsx'), index = False)

<a id="segmentation"></a>

# 3. Text segmentation

[Table of Content](#TOC)

In [ ]:
df_incidents = pd.read_excel(os.path.join(path_to_data_prc, 'source_texts.xlsx'))

In [ ]:
df_incidents.head(3)

## 3.1 Segmentation into paragraphs

In [ ]:
paragraphs = df_incidents.apply(
    func = lambda r: [(r.Doc_id, i, r.title, r.date, r.location, p) for i, p in enumerate(re.sub('(\n)+', '\n', r.text).split('\n'))],
    axis = 1,
)
paragraphs = [p for ps in paragraphs for p in ps]

In [ ]:
df_paragraphs = pd.DataFrame(paragraphs, columns = ['Doc_id', 'Para_id', 'title', 'date', 'location', 'text'])
df_paragraphs.shape

In [ ]:
df_paragraphs.to_excel(os.path.join(path_to_data_prc, 'source_paragraphs.xlsx'), index = False)

## 3.2 Segmentation into sentences

In [ ]:
tokenizer = PunktSentenceTokenizer()

In [ ]:
sentences = df_paragraphs.apply(
    func = lambda r: [(r.Doc_id, r.Para_id, i, r.title, r.date, r.location, s) for i, s in enumerate(tokenizer.tokenize(r.text))],
    axis = 1,
)
sentences = [s for ss in sentences for s in ss]

In [ ]:
df_sentences = pd.DataFrame(sentences, columns = ['Doc_id', 'Para_id', 'Sent_id', 'title', 'date', 'location', 'text'])
df_sentences.shape

In [ ]:
df_sentences.to_excel(os.path.join(path_to_data_prc, 'source_sentences.xlsx'), index = False)

## 3.3 Span extraction

[Table of Content](#TOC)

In [ ]:
def get_span_data(s):
    return (s.text, s.lemma_, s.root.lemma_, s.start, s.end)

In [ ]:
spans = df_sentences.apply(
    func = lambda r: [
        [r.Doc_id, r.Para_id, r.Sent_id, i, r.title, r.date, r.location] + list(get_span_data(s)) 
        for i, s in enumerate(get_maximal_spans(nlp(r.text)))
    ],
    axis = 1,
)
spans = [s for ss in spans for s in ss]

In [ ]:
df_spans = pd.DataFrame(spans, columns = [
    'Doc_id', 'Para_id', 'Sent_id', 'Span_id', 'title', 'date', 'location', 'text', 'lemma', 'root', 'start', 'end',
])
df_spans.shape

In [ ]:
df_spans.to_excel(os.path.join(path_to_data_prc, 'source_spans.xlsx'), index = False)

<a id="bottom"></a>

[Table of content](#TOC)